In [ ]:
from typing import List, Type
from pydantic import BaseModel
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
import json

# ---------------------------
# 1. Define the Pydantic Schema
# ---------------------------
class ParsedRequest(BaseModel):
    intent: str                 # "table", "qualitative", or "in-depth"
    banks: List[str]            # Must match allowed_banks
    quarters: List[str]         # e.g., "1Q2025", "4Q2024"
    metrics: List[str]          # Must match allowed_metrics


# ---------------------------
# 2. Function to Use ChatOllama with Pydantic Schema
# ---------------------------
def parse_with_chatollama(
    llm_model_name: str,
    user_input: str,
    schema: Type[BaseModel],
    allowed_banks: List[str],
    allowed_metrics: List[str]
) -> BaseModel:
    system_prompt = f"""
You are a financial assistant. Your task is to extract structured information from user input and return it in the following JSON format:

{{
  "intent": "table" | "qualitative" | "in-depth",
  "banks": [valid bank names],
  "quarters": ["1Q2025", "4Q2024", "3Q2024"],
  "metrics": [valid metric keys]
}}

Rules:
- Choose "intent" based on context: "table" (structured data), "qualitative" (light summary), "in-depth" (detailed analysis).
- Map any abbreviation or alias to official bank names from this list: {json.dumps(allowed_banks)}
- Extract all mentioned quarters in "1Q2025" format. Include the previous 2 quarters for each.
- Extract only metrics listed here: {json.dumps(allowed_metrics)}.
- Output only the JSON structure as shown above, no explanation or markdown.
"""

    # Load the Ollama model
    llm = ChatOllama(model=llm_model_name)

    # Create and send the prompt
    messages = [
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=user_input.strip())
    ]
    response = llm.invoke(messages)

    # Validate using Pydantic
    try:
        return schema.parse_raw(response.content)
    except Exception as e:
        raise ValueError(f"Failed to parse model output: {e}\nRaw Output:\n{response.content}")


In [ ]:
if __name__ == "__main__":
    allowed_banks = ["JP Morgan Chase", "Bank of America", "Citigroup", "Wells Fargo"]
    allowed_metrics = ["EarningsPerShare", "NetIncome", "TotalRevenue", "ReturnOnEquity"]

    user_input = "I want a table comparing EPS and Net income of JPMC and BOA for Q1 2025"

    result = parse_with_chatollama(
        llm_model_name="llama3",  # Replace with your loaded Ollama model name
        user_input=user_input,
        schema=ParsedRequest,
        allowed_banks=allowed_banks,
        allowed_metrics=allowed_metrics
    )

    print(result.json(indent=2))


In [ ]:
import json

# Validate using Pydantic
try:
    raw = response.content.strip()

    # Optional: clean triple backticks if LLM returns markdown
    if "```" in raw:
        import re
        match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", raw, re.DOTALL)
        if match:
            raw = match.group(1)

    # Step 1: Convert JSON string to Python dict
    parsed_dict = json.loads(raw)

    # Step 2: Validate and convert into a Pydantic object
    parsed_model = schema.parse_obj(parsed_dict)

    # Step 3: Return it (now it has `.model_dump_json()` etc.)
    return parsed_model

except Exception as e:
    raise ValueError(f"Failed to parse model output: {e}\nRaw Output:\n{response.content}")
